## Capstone Project - The Battle of Neighborhoods (Week 2)

### 0. Import Packages

In [1]:
import numpy as np 
import pandas as pd
import json
from geopy.geocoders import Nominatim
import requests 
from pandas import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

### 1. Import Data from last week's project

In [2]:
data = pd.read_csv('taipei_geo.csv', index_col=0)
data

,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,name,1st_categories
0,"Zhongzheng Dist., Taipei City",25.032405,121.519884,Kinfen Braised Pork Rice (金峰魯肉飯),25.032194,121.518534,Taiwanese Restaurant,Taiwanese Restaurant,Food
1,"Datong Dist., Taipei City",25.063424,121.513042,大橋頭老牌筒仔米糕,25.064753,121.511127,Taiwanese Restaurant,Taiwanese Restaurant,Food
2,"Datong Dist., Taipei City",25.063424,121.513042,旗魚新竹米粉,25.067305,121.511002,Taiwanese Restaurant,Taiwanese Restaurant,Food
3,"Datong Dist., Taipei City",25.063424,121.513042,昌吉豬血湯,25.065864,121.516489,Taiwanese Restaurant,Taiwanese Restaurant,Food
4,"Datong Dist., Taipei City",25.063424,121.513042,珠記大橋頭油飯,25.062899,121.514732,Taiwanese Restaurant,Taiwanese Restaurant,Food
...,...,...,...,...,...,...,...,...,...
282,"Banqiao Dist., New Taipei City",25.011865,121.457967,AWHILE. 外兒小館,25.014028,121.460790,Deli / Bodega,Deli / Bodega,Food
283,"Banqiao Dist., New Taipei City",25.011865,121.457967,HiMall麗寶百貨廣場,25.012792,121.462615,Shopping Mall,Shopping Mall,Shop & Service
284,"Banqiao Dist., New Taipei City",25.011865,121.457967,Bon Appétit (朋派),25.011638,121.462537,Buffet,Buffet,Food
286,"Yonghe Dist., New Taipei City",25.008102,121.516745,太原養生會館,25.006819,121.513094,Massage Studio,Massage Studio,Shop & Service


### 2. Onehot data to Find out how often each place appears in each region

In [3]:
taipei_onehot = pd.get_dummies(data['1st_categories'])
taipei_onehot['District'] = data['District']

In [4]:
taipei_group = taipei_onehot.groupby('District').mean().reset_index()
taipei_group

,District,Arts & Entertainment,Food,Outdoors & Recreation,Shop & Service
0,"Banqiao Dist., New Taipei City",0.114286,0.628571,0.085714,0.171429
1,"Datong Dist., Taipei City",0.052632,0.736842,0.052632,0.157895
2,"Da’an Dist., Taipei City",0.027027,0.864865,0.000000,0.108108
3,"Neihu Dist., Taipei City",0.000000,0.818182,0.000000,0.181818
4,"Songshan Dist., Taipei City",0.000000,0.719298,0.070175,0.210526
5,"Wenshan Dist., Taipei City",0.000000,0.739130,0.043478,0.217391
6,"Xinyi Dist., Taipei City",0.000000,0.368421,0.473684,0.157895
7,"Yonghe Dist., New Taipei City",0.000000,0.736842,0.000000,0.263158
8,"Zhongshan Dist., Taipei City",0.000000,0.461538,0.076923,0.461538
9,"Zhongzheng Dist., Taipei City",0.161290,0.806452,0.032258,0.000000


### 3. Kmean clustering 

In [5]:
kclusters = 4
taipei_clustering = taipei_group.drop('District', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(taipei_clustering)
kmeans.labels_

array([3, 3, 0, 0, 3, 3, 1, 3, 2, 0])

### 4. Use the sum of venue frequencies to find the user's preferred administrative district and use Kmean data to find approximate administrative districts

In [6]:
interest = ['Arts & Entertainment','Food']
value = 0
for i in range(len(interest)):
    value = value + taipei_group[interest[i]]
print('Top Frequency:',taipei_group['District'][np.argmax(value)])
print('Recommend district:')
print(taipei_group['District'][kmeans.labels_ == kmeans.labels_[np.argmax(value)]])

Top Frequency: Zhongzheng Dist., Taipei City
Recommend district:
2         Da’an Dist., Taipei City
3         Neihu Dist., Taipei City
9    Zhongzheng Dist., Taipei City
Name: District, dtype: object


### 5. List the number of places in each recommended administrative district for users to choose their favorite administrative district

In [7]:
taipei_count = data.groupby('District')['1st_categories'].value_counts().to_frame()

In [8]:
district = taipei_group['District'][kmeans.labels_ == kmeans.labels_[np.argmax(value)]].values
interest_count = []

for idx in range(len(district)):
    interest_count.append([])
    for idx2 in range(len(interest)):
        try:
            count = taipei_count['1st_categories'][district[idx], interest[idx2]]
        except:
            count = 0
        interest_count[idx].append(count)
district_count = pd.DataFrame(interest_count, columns=interest)
district_count['District'] = district
district_count

,Arts & Entertainment,Food,District
0,1,32,"Da’an Dist., Taipei City"
1,0,18,"Neihu Dist., Taipei City"
2,5,25,"Zhongzheng Dist., Taipei City"
